In [7]:
import sys

sys.path.append("./../../src")

from Dataset import SpeechCommandsDataset
from torch.utils.data import DataLoader
from Transformer import SpeechCommandTransformer, train_transformer
import torch
from torch.optim import AdamW

In [2]:
torch.cuda.empty_cache()

In [ ]:
train_dataset = SpeechCommandsDataset("../../data/train", mode="original")
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6)

test_dataset = SpeechCommandsDataset("../../data/test", mode="original")
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SpeechCommandTransformer(num_classes=len(train_dataset.class_to_idx), embed_dim=256, device=device,
                                 stride=2).to(device)
optimizer = AdamW(model.parameters(), lr=0.00005, weight_decay=0.001)

train_transformer(train_loader, test_loader, model=model, num_epochs=10, device=device, optimizer=optimizer)

Epoch 1/10:  19%|█▉        | 621/3292 [05:12<23:09,  1.92it/s, acc=6.75, loss=3.36]